In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime
from pyspark.sql import Window
from pyspark.sql.functions import length, col, rowNumber, regexp_extract, concat, format_string

In [2]:
def latest(df, ids, timestamp_cols):
    return df \
        .withColumn('_row', rowNumber().over(
            Window \
                .partitionBy(*[col(id) for id in ids]) \
                .orderBy(*[col(timestamp_col).desc() for timestamp_col in timestamp_cols]))) \
        .filter(col('_row') == 1) \
        .drop('_row')

In [3]:
def latest_contcur(df, ids):
    return latest(df, ids, ['contcur'])

In [4]:
def latest_dat_ref_carga(df, ids):
    return latest(df, ids, ['dat_ref_carga'])

In [18]:
base_contestacao_df = sqlContext.table('arq_fraude.base_contestacao')

In [93]:
base_contestacao_latest_df = latest_dat_ref_carga(
    base_contestacao_df \
        .withColumn('numconta', base_contestacao_df.numconta.cast('long').cast('string')),
    ['codent', 'numagencia', 'numconta', 'dttransacao', 'valorfraude'])

In [99]:
compras_contestadas_pred = \
    (base_contestacao_latest_df.canal == 'AUTO ATENDIMENTO') & \
    (base_contestacao_latest_df.descricaotransacao == 'Compras')

In [104]:
contalynx_ok_pred = length(base_contestacao_df.contalynx) == 19

In [110]:
# .filter(contalynx_ok_pred)
compras_contestadas_df = base_contestacao_latest_df \
    .filter(compras_contestadas_pred) \
    .select(
        base_contestacao_latest_df.codent,
        base_contestacao_latest_df.numagencia,
        base_contestacao_latest_df.numconta,
        concat(
            base_contestacao_latest_df.codent,
            base_contestacao_latest_df.numagencia,
            format_string('%012d', base_contestacao_latest_df.numconta.cast('long'))) \
            .alias('ctacargo'),
        base_contestacao_latest_df.cartao,
        base_contestacao_latest_df.dttransacao,
        base_contestacao_latest_df.valorfraude)

In [117]:
mpdt014_df = sqlContext.table('mp.mpdt014')

In [118]:
dat_ref_carga = mpdt014_df.groupBy().agg({'dat_ref_carga': 'max'}).collect()[0][0]
ultima_carga_pred = mpdt014_df.dat_ref_carga == dat_ref_carga

In [119]:
mpdt014_latest_df = latest_contcur(
    mpdt014_df.filter(ultima_carga_pred & (mpdt014_df.indcta == 'P')),
    ['codent', 'centalta', 'cuenta', 'ctacargo'])

In [122]:
contas_contratos_df = mpdt014_latest_df \
    .select(
        mpdt014_latest_df.codent,
        mpdt014_latest_df.centalta,
        mpdt014_latest_df.cuenta,
        mpdt014_latest_df.ctacargo,
        mpdt014_latest_df.fecalta,
        mpdt014_latest_df.fecbaja)

In [140]:
compras_contestadas_contratos_df = compras_contestadas_df \
    .withColumnRenamed('ctacargo', 'ctacargo_gen') \
    .join(
        contas_contratos_df \
            .withColumnRenamed('codent', 'codent_014'),
        (col('ctacargo_gen') == contas_contratos_df.ctacargo) &
        (compras_contestadas_df.dttransacao >= contas_contratos_df.fecalta) &
        (contas_contratos_df.fecbaja.isNull() | (compras_contestadas_df.dttransacao <= contas_contratos_df.fecbaja)),
        'left_outer')

In [142]:
compras_contestadas_contratos_df.write.mode('overwrite').parquet('hdfs:/user/x181471/compras_contestadas_contratos')

In [7]:
compras_contestadas_contratos_df = sqlContext.read.parquet('hdfs:/user/x181471/compras_contestadas_contratos')

In [149]:
mpdt090_df = sqlContext.table('mp.mpdt090')

In [150]:
data_attr = 'fecfac'
valor_attr = 'impdiv'
mpdt090_attrs = ['codent', 'centalta', 'cuenta', 'siaidcd', 'fecfac', 'impdiv', 'pan', 'impfac', 'impliq', 'codcom', 'nomcomred', 'codpais', 'nompob', 'fecconta']

In [151]:
mpdt090_latest_df = latest_contcur(
    mpdt090_df.filter(
        mpdt090_df.codcom.isNotNull() & (mpdt090_df.codcom != '')),
    ['codent', 'centalta', 'cuenta', 'siaidcd', data_attr, valor_attr]) \
    .select(*mpdt090_attrs) \
    .withColumnRenamed('codcom', 'codcom_090') \
    .withColumnRenamed('nomcomred', 'nomcomred_090') \
    .withColumnRenamed('impdiv', 'impdiv_090') \
    .withColumnRenamed('pan', 'pan_090') \
    .withColumnRenamed('fecconta', 'fecconta_090')

In [156]:
mpdt004_df = sqlContext.table('mp.mpdt004')

In [157]:
mpdt004_latest_df = latest_contcur(mpdt004_df, ['codent', 'centalta', 'cuenta', 'siaidcd'])

In [158]:
mpdt004_df_attrs = ['codent', 'centalta', 'cuenta', 'siaidcd', 'pan', 'imptrn', 'impdiv', 'fectrn', 'hortrn', 'idterm', 'codcom', 'nomcom', 'localidad', 'fecconta']

In [159]:
autorizacoes_debito_df = mpdt004_latest_df \
    .filter(
        (mpdt004_latest_df.inddebcre == '2') &
        mpdt004_latest_df.codcom.isNotNull() &
        (mpdt004_latest_df.codcom != '')) \
    .select(*mpdt004_df_attrs) \
    .withColumnRenamed('codcom', 'codcom_004') \
    .withColumnRenamed('nomcom', 'nomcom_004') \
    .withColumnRenamed('impdiv', 'impdiv_004') \
    .withColumnRenamed('pan', 'pan_004') \
    .withColumnRenamed('fecconta', 'fecconta_004')

In [163]:
debito_autorizado_df = mpdt090_latest_df.join(
    autorizacoes_debito_df,
    (mpdt090_latest_df.codent == autorizacoes_debito_df.codent) &
    (mpdt090_latest_df.centalta == autorizacoes_debito_df.centalta) &
    (mpdt090_latest_df.cuenta == autorizacoes_debito_df.cuenta) &
    (mpdt090_latest_df.siaidcd == autorizacoes_debito_df.siaidcd),
    'inner') \
    .drop(autorizacoes_debito_df.codent) \
    .drop(autorizacoes_debito_df.centalta) \
    .drop(autorizacoes_debito_df.cuenta) \
    .drop(autorizacoes_debito_df.siaidcd)

In [165]:
debito_autorizado_df.write.mode('overwrite').parquet('hdfs:/user/x181471/debito_autorizado')
# 40 mins, 48 execs

In [8]:
# 31.1 G  93.2 G
debito_autorizado_df = sqlContext.read.parquet('hdfs:/user/x181471/debito_autorizado')

In [22]:
debito_df = debito_autorizado_df \
    .join(
        compras_contestadas_contratos_df,
        (compras_contestadas_contratos_df.codent == debito_autorizado_df.codent) &
        (compras_contestadas_contratos_df.centalta == debito_autorizado_df.centalta) &
        (compras_contestadas_contratos_df.cuenta == debito_autorizado_df.cuenta) &
        (compras_contestadas_contratos_df.valorfraude == debito_autorizado_df.impdiv_090) &
        (compras_contestadas_contratos_df.dttransacao == debito_autorizado_df.fecfac),
        'left_outer') \
    .drop(debito_autorizado_df.codent) \
    .drop(debito_autorizado_df.centalta) \
    .drop(debito_autorizado_df.cuenta)

In [17]:
mpdt343_df = sqlContext.table('mp.mpdt343')

In [19]:
mpdt343_latest_df = latest(mpdt343_df, ['cd_idef_sia'], ['dat_ref_carga', 'dh_ulti_atlz'])

In [11]:
mpdt343_latest_df \
    .select(*[col(attr) for attr in ['cd_idef_sia', 'in_senh', 'in_codi_segr', 'in_crto_chip', 'nr_crto', 'dh_tran', 'vl_orig_tran', 'cd_empr_grup_sntd', 'nr_cntr_crto', 'cd_idef_tran']]) \
    .write.mode('overwrite').parquet('hdfs:/user/x181471/modo_entrada')
# 7.9 min, 180 execs

In [12]:
mpdt343_latest_df = sqlContext.read.parquet('hdfs:/user/x181471/modo_entrada')